Импорты

In [1]:
from datetime import datetime

from requester import Requester

Проверяем работоспособность эластика

In [2]:
es_url = 'http://localhost:9200'
es = Requester(url=es_url)

In [3]:
es.get()

{'name': 'MacBook-Air-Konstantin.local',
 'cluster_name': 'elasticsearch_konstantinivanov',
 'cluster_uuid': '2bL4LnIpRRSXbzu2eM_GRQ',
 'version': {'number': '7.5.2',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': '8bec50e1e0ad29dad5653712cf3bb580cd1afcdf',
  'build_date': '2020-01-15T12:11:52.313576Z',
  'build_snapshot': False,
  'lucene_version': '8.3.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

## Индексация

[Проиндексируем](https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-create-index.html) один документ (в нашем случае  - это чек)

In [4]:
doc = {
    "text": "Молоко ДВД 0.97л",
    "type": "молоко",
    "brand": "Домик в Деревне",
    "published_at": datetime.now().strftime("%Y-%m-%d")
}

Положим документ в подходящее место  
Структура данных 
* __index_ - по смыслу База Данных, например, **receipts**
* __type_ - по смыслу Таблица, например, **raw**
* __id_ - айди документа, например, **1**

data
* документ

In [5]:
es.put(path='receipts/_doc/1', data=doc)

{'_index': 'receipts',
 '_type': '_doc',
 '_id': '1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

Посмотрим на созданный индекс

In [6]:
es.get(path='_cat/indices')

yellow open receipts xCImHOQfTKKGkMIaQEnfiw 1 1 1 0 5.1kb 5.1kb



Посмотрим на созданный маппинг

In [7]:
es.get(path='receipts/_mapping?pretty')

{'receipts': {'mappings': {'properties': {'brand': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'published_at': {'type': 'date'},
    'text': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'type': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}}}

## Запросы

[Запрос](https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-get-index.html) документов по их id

In [8]:
es.get(path='receipts/_doc/1')

{'_index': 'receipts',
 '_type': '_doc',
 '_id': '1',
 '_version': 1,
 '_seq_no': 0,
 '_primary_term': 1,
 'found': True,
 '_source': {'text': 'Молоко ДВД 0.97л',
  'type': 'молоко',
  'brand': 'Домик в Деревне',
  'published_at': '2020-02-17'}}

Или только документ без дополнительной информации

In [9]:
es.get(path='receipts/_doc/1/_source')

{'text': 'Молоко ДВД 0.97л',
 'type': 'молоко',
 'brand': 'Домик в Деревне',
 'published_at': '2020-02-17'}

Проиндексируем еще несколько документов

In [10]:
doc = {
    "text": "Молочко для тела GARNIER",
    "type": "молочко",
    "brand": "GARNIER",
    "published_at": datetime.now().strftime("%Y-%m-%d")
}
es.put(path='receipts/_doc/2', data=doc)

doc = {
    "text": "Крем для тела GARNIER",
    "type": "крем",
    "brand": "GARNIER",
    "published_at": datetime.now().strftime("%Y-%m-%d")
}
es.put(path='receipts/_doc/3', data=doc)

{'_index': 'receipts',
 '_type': '_doc',
 '_id': '3',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 2,
 '_primary_term': 1}

Посмотрим на индекс

In [11]:
es.get(path='_cat/indices')

yellow open receipts xCImHOQfTKKGkMIaQEnfiw 1 1 3 0 5.1kb 5.1kb



[**Boolean query**](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html) - один из основных типов запросов
* _must_ - "обязательные условия", объединяются через И, влияют на __score_
* _filter_ - фильтры, не влияют на __score_
* _should_ - "желательные условия", объединяются через ИЛИ, влияют на __score_
* _must_not_ - условия для исключений, обнуляют __score_

Must

In [12]:
query = {
    "_source": ["text", "published_at"],
    "query": {
        "bool": {
            "must": [{
                "match": {
                    "text": "молоко"
                }
            }]
        }
    }
}

es.get(path='receipts/_doc/_search', data=query)

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.0596458,
  'hits': [{'_index': 'receipts',
    '_type': '_doc',
    '_id': '1',
    '_score': 1.0596458,
    '_source': {'text': 'Молоко ДВД 0.97л', 'published_at': '2020-02-17'}}]}}

In [13]:
query = {
    "_source": ["text", "published_at"],
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "text": "молоко"
                    }
                },
                {
                    "match": {
                        "text": "молочко"
                    }
                },
            ]
        }
    }
}

es.get(path='receipts/_doc/_search', data=query)

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

Filter

In [14]:
query = {
    "_source": ["text", "published_at"],
    "query": {
        "bool": {
            "filter": [{
                "match": {
                    "text": "молоко"
                }
            }]
        }
    }
}

es.get(path='receipts/_doc/_search', data=query)

{'took': 6,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.0,
  'hits': [{'_index': 'receipts',
    '_type': '_doc',
    '_id': '1',
    '_score': 0.0,
    '_source': {'text': 'Молоко ДВД 0.97л', 'published_at': '2020-02-17'}}]}}

Should

In [15]:
query = {
    "_source": ["text", "published_at"],
    "query": {
        "bool": {
            "should": [
                {
                    "match": {
                        "text": "молоко"
                    }
                },
                {
                    "match": {
                        "text": "молочко"
                    }
                },
            ]
        }
    }
}

es.get(path='receipts/_doc/_search', data=query)

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.0596458,
  'hits': [{'_index': 'receipts',
    '_type': '_doc',
    '_id': '1',
    '_score': 1.0596458,
    '_source': {'text': 'Молоко ДВД 0.97л', 'published_at': '2020-02-17'}},
   {'_index': 'receipts',
    '_type': '_doc',
    '_id': '2',
    '_score': 0.94566005,
    '_source': {'text': 'Молочко для тела GARNIER',
     'published_at': '2020-02-17'}}]}}

[**Match query**](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html) - полнотекстовый поиск, который мы использовали выше  
(основные параметры)
* _query_
* _analyzer_
* _operator_

## Анализаторы

По умолчанию эластик использует [стандартный анализатор](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-standard-analyzer.html)

In [16]:
text = {
    "analyzer": "standard",
    "text": "Ёмкость для воды"
}

es.post(path='_analyze', data=text)

{'tokens': [{'token': 'ёмкость',
   'start_offset': 0,
   'end_offset': 7,
   'type': '<ALPHANUM>',
   'position': 0},
  {'token': 'для',
   'start_offset': 8,
   'end_offset': 11,
   'type': '<ALPHANUM>',
   'position': 1},
  {'token': 'воды',
   'start_offset': 12,
   'end_offset': 16,
   'type': '<ALPHANUM>',
   'position': 2}]}

Есть и стандартный [русский анализатор](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-lang-analyzer.html#russian-analyzer)

In [17]:
text = {
    "analyzer": "russian",
    "text": "Ёмкость для воды"
}

es.post(path='_analyze', data=text)

{'tokens': [{'token': 'ёмкост',
   'start_offset': 0,
   'end_offset': 7,
   'type': '<ALPHANUM>',
   'position': 0},
  {'token': 'вод',
   'start_offset': 12,
   'end_offset': 16,
   'type': '<ALPHANUM>',
   'position': 2}]}

Напишем свой анализатор

In [18]:
analyzer_custom = {
    "filter": {
        "russian_stop": {
            "type":       "stop",
            "stopwords":  "_russian_" 
        },
        "russian_stemmer": {
            "type":       "stemmer",
            "language":   "russian"
        },
    },
    "char_filter": {
        "e_char_filter": {
            "type": "mapping",
            "mappings": [
                "ё => е",
                "Ё => Е"
            ]
        }
    },
    "analyzer": {
        "analyzer_cus": {
            "tokenizer":  "standard",
            "char_filter": [
                "e_char_filter"
            ],
            "filter": [
                "lowercase",
                "russian_stop",
                "russian_stemmer",
            ]
        }
    }
}

Создадим индекс с новым анализатором

In [19]:
settings = {
    "settings": {
        "analysis": analyzer_custom
    },
    "mappings": {
        "properties": {
            "text": {
                "type": "text",
                "analyzer": "analyzer_cus"
            },
            "type": {
                "type": "text",
                "analyzer": "analyzer_cus"
            },
            "brand": {
                "type": "text",
                "analyzer": "analyzer_cus"
            },
            "published_at": {
                "type": "date"
            }
        }
    }
}

es.put(path='receipts_cus', data=settings)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'receipts_cus'}

In [20]:
text = {
    "analyzer": "analyzer_cus",
    "text": "Ёмкость для воды"
}

es.post(path='receipts_cus/_analyze', data=text)

{'tokens': [{'token': 'емкост',
   'start_offset': 0,
   'end_offset': 7,
   'type': '<ALPHANUM>',
   'position': 0},
  {'token': 'вод',
   'start_offset': 12,
   'end_offset': 16,
   'type': '<ALPHANUM>',
   'position': 2}]}

Положим в него документы

In [21]:
doc = {
    "text": "Емкость для порошка",
    "type": "емкость",
    "brand": "no_name",
    "published_at": datetime.now().strftime("%Y-%m-%d")
}
es.put(path='receipts_cus/_doc/1', data=doc)

doc = {
    "text": "Ёмкость для воды",
    "type": "емкость",
    "brand": "no_name",
    "published_at": datetime.now().strftime("%Y-%m-%d")
}
es.put(path='receipts_cus/_doc/2', data=doc)

{'_index': 'receipts_cus',
 '_type': '_doc',
 '_id': '2',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

Сделаем **match** запрос

In [22]:
query = {
    "_source": ["text", "published_at"],
    "query": {
        "match": {
            "text": {
                "query": "емкость"
            }
        }
    }
}

es.get(path='receipts_cus/_doc/_search', data=query)

{'took': 8,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 0.18232156,
  'hits': [{'_index': 'receipts_cus',
    '_type': '_doc',
    '_id': '1',
    '_score': 0.18232156,
    '_source': {'text': 'Емкость для порошка', 'published_at': '2020-02-17'}},
   {'_index': 'receipts_cus',
    '_type': '_doc',
    '_id': '2',
    '_score': 0.18232156,
    '_source': {'text': 'Ёмкость для воды', 'published_at': '2020-02-17'}}]}}

[**Query string**](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html) - Полнотекстовый поиск c использованием логических операций

In [23]:
query = {
    "query": {
        "query_string": {
            "query": "ёмкость -вода",
            "fields": [
                "text"
            ]
        }
    }
}
es.post(path='receipts_cus/_doc/_search', data=query)

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.18232156,
  'hits': [{'_index': 'receipts_cus',
    '_type': '_doc',
    '_id': '1',
    '_score': 0.18232156,
    '_source': {'text': 'Емкость для порошка',
     'type': 'емкость',
     'brand': 'no_name',
     'published_at': '2020-02-17'}}]}}

## Удаление индекса

In [24]:
es.get(path='_cat/indices')

yellow open receipts     xCImHOQfTKKGkMIaQEnfiw 1 1 3 0 10.2kb 10.2kb
yellow open receipts_cus q5wPjFksS_qZ78VqGTUFdg 1 1 2 0  3.8kb  3.8kb



Удалим индекс

In [25]:
es.delete('receipts')
es.delete('receipts_cus')

{'acknowledged': True}

In [26]:
es.get(path='_cat/indices')

## Допы

Что посмотреть про эластик  
* [Основы Elasticsearch / Хабр](https://habr.com/ru/post/280488/) - использовал отсюда идеи для семинара  
* [Elasticsearch tutorial for beginners using Python / Medium](https://medium.com/naukri-engineering/elasticsearch-tutorial-for-beginners-using-python-b9cb48edcedc)
* [Документация Elasticsearch](https://www.elastic.co/guide/en/elasticsearch/reference/7.5/rest-apis.html)

Разное
* [Convert curl syntax to Python](https://curl.trillworks.com)